

# **Predicción con uKNN**


# Índice

>[Índice](#scrollTo=2j02H66f87eV)

>[Descargando la información](#scrollTo=9qKTQ2V5VKSX)

>[Preparar entorno](#scrollTo=P3K5VPZoGSYX)

>[UserKNN](#scrollTo=2e0ZeZXzMhU3)

>>[Generar una lista de recomendaciones para un usuario](#scrollTo=dIVXNsQ8CCJV)

>>>[Evaluar en base a error de prediccion (RMSE y MAE)](#scrollTo=g55SDLmqCpUo)



# Descargando la información


In [1]:
import pandas as pd

train_file = pd.read_csv('../data/training_set.csv', sep=',', header=0)
train_file.head()

,userID,itemID,styleID,rating
0,401,32780,12224,3.0
1,7548,21688,9020,3.5
2,7663,1521,568,4.0
3,1357,3824,1417,3.5
4,1361,727,263,4.5


In [2]:
validation_file = pd.read_csv('../data/validation_set.csv', sep=',', header=0)
validation_file.head()

,userID,itemID,styleID,rating
0,1040,4920,1315,4.0
1,1306,7348,1199,4.0
2,932,4073,1199,4.0
3,6735,16074,1199,4.5
4,197,39299,3268,4.0


## Preparando entorno

In [3]:
!pip install pyreclab

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [4]:
import pyreclab
import numpy as np
import json
import os
import heapq
import math
import matplotlib.pyplot as plt
from sklearn.metrics import pairwise_distances
from sklearn.decomposition import PCA
from io import BytesIO
import pickle
import pandas as pd
import time

## UserKNN


In [5]:
# Declaración del objeto recomendador
myUserKnn = pyreclab.UserKnn(dataset='../data/training_set.csv',
                             dlmchar=b',',
                             header=False,
                             usercol=0,
                             itemcol=1,
                             ratingcol=3)

## Análisis de sensibilidad - K vecinos

Utilizando **Pearson**

In [6]:
k_values = [10, 30, 50, 70, 100, 130] # definir distintos valores de vecinos (k)
mae_values = [] # para almacenar resultados de MAE
rmse_values = [] # para almacenar valores de RMSE

for k in k_values:
  myUserKnn.train(k,'pearson')
  predlist, mae, rmse = myUserKnn.test( input_file = '../data/validation_set.csv',dlmchar = b',', header = False, usercol = 0, itemcol = 1, ratingcol = 3, output_file = 'predictions.csv' )
  rmse_values.append(rmse)
  mae_values.append(mae)

print('RMSE:', rmse_values)
print('MAE:', mae_values)

on line 30648


 in training set

n training set
W

arning: item id '34153' was not included in training set

t included in training set


KeyboardInterrupt: SIGINT received

Utilizando **Coseno**

In [ ]:
k_values = [10, 30, 50, 70, 100, 130] # definir distintos valores de vecinos (k)
cosine_mae_values = [] # para almacenar resultados de MAE
cosine_rmse_values = [] # para almacenar valores de RMSE

for k in k_values:
  myUserKnn.train(k,'cosine')
  predlist, mae, rmse = myUserKnn.test( input_file = '../data/validation_set.csv',dlmchar = b',', header = False, usercol = 0, itemcol = 1, ratingcol = 3, output_file = 'predictions.csv' )
  cosine_rmse_values.append(rmse)
  cosine_mae_values.append(mae)

print('RMSE:', cosine_rmse_values)
print('MAE:', cosine_mae_values)

### Evaluar en base a error de prediccion RMSE y MAE

Para pearson:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(k_values, rmse_values, 'r-')
plt.plot(k_values, rmse_values, 'bo')
plt.show()

In [ ]:
plt.plot(k_values, mae_values, 'y-')
plt.plot(k_values, mae_values, 'go')
plt.show()

Para Coseno:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(k_values, cosine_rmse_values, 'r-')
plt.plot(k_values, cosine_rmse_values, 'bo')
plt.show()

In [ ]:
plt.plot(k_values, cosine_mae_values, 'y-')
plt.plot(k_values, cosine_mae_values, 'go')
plt.show()

## Entrenando modelo


In [6]:
myUserKnn.train(100,'cosine')

## Rating


In [7]:
rating_test = pd.read_csv('../templates/rating_template_fill.csv', sep=';', header=0)
rating_test.head()

,itemID,styleID,Rating,userID
0,39289,14756,NaN,715
1,3939,1417,NaN,4944
2,47695,14879,NaN,4890
3,7348,1199,NaN,5324
4,37403,1199,NaN,5544


In [8]:
# Función para obtener el promedio de ratings de un usuario en caso de que el item no esté en training
def user_average_rating(user_id, train_ratings):
    user_ratings = train_ratings[train_ratings['userID'] == user_id]['rating']
    if len(user_ratings) > 0:
        return user_ratings.mean()
    else:
        return train_ratings['rating'].mean()

# Lista de ítems en training
training_items = train_file['itemID'].unique()

In [9]:
for index, row in rating_test.iterrows():
    user_id = str(int(row.iloc[3]))
    item_id = str(int(row.iloc[0]))

    if int(item_id) not in training_items:
      rating = user_average_rating(user_id, train_file)
    else:
      rating = myUserKnn.predict(user_id, item_id)
    rating_test.at[index, 'Rating'] = rating


In [10]:
rating_test.head()

,itemID,styleID,Rating,userID
0,39289,14756,4.610866,715
1,3939,1417,3.548518,4944
2,47695,14879,4.037550,4890
3,7348,1199,4.437984,5324
4,37403,1199,4.229623,5544


In [11]:
ruta = os.path.join('../predictions', 'rating_uknn.csv')
rating_test.to_csv(ruta, index=False)